In [12]:
import pandas as pd
import os
import re

# ============================================================================
# CHANGE THIS TO PROCESS DIFFERENT FOLDER
folder_num = 12  # Change this to 2, 3, 4, ... 26
# ============================================================================

# Base directories
sl512_npu = 'Encoder-BS32-SL512/npu'
sl256_pim = 'Encoder-BS32-SL256/pim'
sl512_pim_output = 'Encoder-BS32-SL512/pim'

# Create output directory if it doesn't exist
os.makedirs(sl512_pim_output, exist_ok=True)

print("="*80)
print(f"Creating SL512 PIM data for Folder {folder_num}")
print("="*80)
print()

folder_name = str(folder_num)

# Input files
npu_txt = os.path.join(sl512_npu, folder_name, 'SA_stage_E.txt')

# Determine which SL256 PIM folder to use (cycling through 2-9)
sl256_folder = ((folder_num - 2) % 8) + 2
pim_txt_256 = os.path.join(sl256_pim, str(sl256_folder), 'SA_stage_E.txt')

# Output location
output_folder = os.path.join(sl512_pim_output, folder_name)
os.makedirs(output_folder, exist_ok=True)
output_txt = os.path.join(output_folder, 'SA_stage_E.txt')

# Check if input files exist
if not os.path.exists(npu_txt):
    print(f"ERROR: NPU file not found: {npu_txt}")
elif not os.path.exists(pim_txt_256):
    print(f"ERROR: SL256 PIM file not found: {pim_txt_256}")
else:
    print(f"Input NPU: {npu_txt}")
    print(f"Input PIM (SL256 folder {sl256_folder}): {pim_txt_256}")
    print(f"Output: {output_txt}")
    print()
    
    # Read NPU SL512 data
    with open(npu_txt, 'r') as f:
        npu_content = f.read()
    
    expert_data = []
    for line in npu_content.split('\n'):
        if not line.strip() or '=' in line or 'Expert Number' in line or 'Expert Operations' in line:
            continue
        
        parts = line.split()
        if len(parts) >= 6:
            try:
                expert_num = int(parts[0])
                fc1_npu = int(parts[2])
                gelu_npu = int(parts[3])
                fc2_npu = int(parts[4])
                
                # Apply transformations
                fc1_pim = fc1_npu * 64
                fc2_pim = fc2_npu * 64
                gelu_pim = gelu_npu + 69
                total_compute = fc1_pim + gelu_pim + fc2_pim
                
                expert_data.append({
                    'Expert Number': expert_num,
                    'fc1': fc1_pim,
                    'gelu': gelu_pim,
                    'fc2': fc2_pim,
                    'total compute': total_compute
                })
            except (ValueError, IndexError):
                continue
    
    # Read SL256 PIM activation movements
    with open(pim_txt_256, 'r') as f:
        pim256_content = f.read()
    
    activation_1 = 0
    activation_2 = 0
    
    for line in pim256_content.split('\n'):
        if 'activation_movement_1' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_1 = int(match.group(1)) * 2  # Multiply by 2
        elif 'activation_movement_2' in line and 'cycles' in line:
            match = re.search(r'(\d+)\s+cycles', line)
            if match:
                activation_2 = int(match.group(1)) * 2  # Multiply by 2
    
    # Create DataFrame
    df = pd.DataFrame(expert_data)
    
    # Write output file
    with open(output_txt, 'w') as f:
        f.write("PIM Expert Operations - Total Cycles\n")
        f.write("Note: 2 activation movements (fc1 and fc2)\n\n")
        
        # Write activation movements
        f.write("--- Activation Movements ---\n")
        f.write(f"layer0.ffn.activation_movement_1: {activation_1} cycles\n")
        f.write(f"layer0.ffn.activation_movement_2: {activation_2} cycles\n")
        f.write(f"Total activation movement cycles: {activation_1 + activation_2}\n")
        f.write("----------------------------\n\n")
        
        f.write("="*80 + "\n")
        f.write(df.to_string(index=False))
    
    print("="*80)
    print("✓ SUCCESS!")
    print("="*80)
    print(f"Created: {output_txt}")
    print(f"Experts processed: {len(expert_data)}")
    print(f"Activation 1: {activation_1:,}")
    print(f"Activation 2: {activation_2:,}")
    print(f"Total activation: {activation_1 + activation_2:,}")


Creating SL512 PIM data for Folder 12

Input NPU: Encoder-BS32-SL512/npu/12/SA_stage_E.txt
Input PIM (SL256 folder 4): Encoder-BS32-SL256/pim/4/SA_stage_E.txt
Output: Encoder-BS32-SL512/pim/12/SA_stage_E.txt

✓ SUCCESS!
Created: Encoder-BS32-SL512/pim/12/SA_stage_E.txt
Experts processed: 64
Activation 1: 1,150,536
Activation 2: 1,145,192
Total activation: 2,295,728
